# PCA原理

In [13]:
import numpy as np

# 假设的数据集 X
X = np.array([[1, 2,4], [3, 4,4], [5, 6,8]])

# 步骤 1: 中心化数据
X_meaned = X - np.mean(X, axis=0)

# 步骤 2: 计算协方差矩阵
cov_matrix = np.cov(X_meaned, rowvar=False)

# 步骤 3: 计算协方差矩阵的特征值和特征向量
'''
在 NumPy 的 np.linalg.eig 函数中，计算出的特征值(eigen_values)和特征向量(eigen_vectors)是一一对应的。
也就是说，eigen_values 数组中的第 i 个元素对应于 eigen_vectors 矩阵中的第 i 列。
每个特征向量都是对应其特征值的特征空间中的一个向量。
'''
eigen_values, eigen_vectors = np.linalg.eig(cov_matrix)

# 步骤 4: 对特征值排序，并找到对应的特征向量
'''
这个流程的步骤大致如下：

计算特征值和特征向量： 通过对协方差矩阵进行特征分解，我们可以得到特征值和特征向量。

特征值排序： 将计算得到的特征值从大到小排序。
np.argsort(eigen_values)[::-1]这段代码首先使用np.argsort函数找到将特征值排序后的索引，
然后通过[::-1]将索引数组翻转，得到一个从大到小排序的索引数组。

对应特征向量排序： 根据特征值的排序结果，将特征向量按照对应的顺序重新排列。
eigen_vectors[:, sorted_index]这段代码选取所有特征向量，并使用sorted_index索引数组来按照特征值的降序对这些特征向量进行排序。
'''
sorted_index = np.argsort(eigen_values)[::-1]
sorted_eigenvalue = eigen_values[sorted_index]
sorted_eigenvectors = eigen_vectors[:, sorted_index]

# 步骤 5: 选择前2个主成分
n_components = 2
eigenvector_subset = sorted_eigenvectors[:, 0:n_components]

# 步骤 6: 转换数据
X_reduced = np.dot(eigenvector_subset.transpose(), X_meaned.transpose()).transpose()

# 输出结果
sorted_eigenvalue, eigenvector_subset, X_reduced


(array([ 1.24785319e+01,  8.54801409e-01, -1.07340525e-15]),
 array([[-0.55439511, -0.43891465],
        [-0.55439511, -0.43891465],
        [-0.62071905,  0.78403308]]),
 array([[ 3.04520582,  0.71028115],
        [ 0.82762539, -1.04537744],
        [-3.87283121,  0.33509629]]))

在 TensorFlow 和 PyTorch 的 PCA 实现中，返回的结果主要有两部分：`X_reduced` 和 `components`。

1. **`X_reduced`**：这是经过主成分分析降维后的数据。它包含了原始数据在选择的主成分上的投影，从而实现了数据的降维。这部分数据通常用于后续的数据分析、可视化或作为机器学习模型的输入特征。在上述例子中，`X_reduced` 是通过将原始的中心化数据点投影到选定的主成分（即特征向量）上得到的，这些主成分是根据它们的特征值从大到小选择的，以保留最多的数据方差。

2. **`components`**：这部分是选定的主成分对应的特征向量，通常按照它们对应的特征值的大小排序。在 PCA 中，每个特征向量代表了数据中的一个方向，而特征值则表示该方向的方差大小。因此，`components` 实际上定义了新的坐标空间，其中的坐标轴是按照数据方差从大到小排列的。在许多应用中，这些主成分用于理解数据的主要变化方向，或者用于构建新的特征空间，以便于数据分析或建模。

总结来说，`X_reduced` 提供了降维后的数据表示，而 `components` 提供了构成这个新数据空间的基向量（即主成分）。这两者一起帮助我们从原始数据中抽象出重要的特征，并且在减少数据复杂性的同时保留了最关键的信息。

In [14]:
import tensorflow as tf

def pca_tensorflow(X, n_components=2):
    # 中心化数据
    X_meaned = X - tf.reduce_mean(X, axis=0)

    # 计算协方差矩阵
    cov_matrix = tf.matmul(X_meaned, X_meaned, transpose_a=True) / (tf.cast(tf.shape(X)[0], tf.float32) - 1)

    # 计算协方差矩阵的特征值和特征向量
    eigen_values, eigen_vectors = tf.linalg.eigh(cov_matrix)

    # 逆序排序特征值和特征向量
    idx = tf.argsort(eigen_values, direction='DESCENDING')
    '''
    tf.gather 是 TensorFlow 中的一个操作，用于根据索引从输入张量中收集切片。
    简而言之，tf.gather 允许你从一个张量中选择特定的元素或切片，并根据提供的索引数组重新组合它们。
    '''
    sorted_eigen_vectors = tf.gather(eigen_vectors, idx, axis=1)


    # 选择前n个主成分
    components = sorted_eigen_vectors[:, :n_components]

    # 转换数据
    X_reduced = tf.matmul(X_meaned, components)

    return X_reduced, components

# 示例数据
X_tf = tf.constant([[1, 2, 4], [3, 4, 4], [5, 6, 8]], dtype=tf.float32)

# 执行PCA
X_reduced_tf, components_tf = pca_tensorflow(X_tf, n_components=2)
print(X_reduced_tf, components_tf)

tf.Tensor(
[[-3.0452063   0.7102812 ]
 [-0.82762563 -1.0453775 ]
 [ 3.8728318   0.33509594]], shape=(3, 2), dtype=float32) tf.Tensor(
[[ 0.55439526 -0.43891522]
 [ 0.5543952  -0.43891415]
 [ 0.62071913  0.78403306]], shape=(3, 2), dtype=float32)


In [15]:
import torch

def pca_pytorch(X, n_components=2):
    # 中心化数据
    X_meaned = X - torch.mean(X, dim=0)

    # 计算协方差矩阵
    cov_matrix = torch.mm(X_meaned.t(), X_meaned) / (X.shape[0] - 1)

    # 计算协方差矩阵的特征值和特征向量
    eigen_values, eigen_vectors = torch.linalg.eigh(cov_matrix, UPLO='U')

    # 逆序排序特征值和特征向量
    '''
    对 eigen_values（一个包含特征值的数组）进行排序，并返回降序排序后的索引数组。
    这个数组的顺序决定了如何根据特征值的大小（从大到小）重新排列特征向量。
    '''
    idx = eigen_values.argsort(descending=True) 
    '''
    使用前一步得到的索引数组 idx 来选择 eigen_vectors 中的列，即特征向量。
    这里 : 表示选择所有行，而 idx 决定了选择哪些列。
    因为 idx 是按照特征值大小降序排列的索引，所以这里选出的特征向量也会按照对应特征值的大小降序排列。
    '''
    sorted_eigen_vectors = eigen_vectors[:, idx]

    # 选择前n个主成分
    components = sorted_eigen_vectors[:, :n_components]

    # 转换数据
    X_reduced = torch.mm(X_meaned, components)

    return X_reduced, components

# 示例数据
X_torch = torch.tensor([[1.0, 2.0, 4.0], [3.0, 4.0, 4.0], [5.0, 6.0, 8.0]])

# 执行PCA
X_reduced_torch, components_torch = pca_pytorch(X_torch, n_components=2)

print(X_reduced_torch, components_torch)

tensor([[-3.0452, -0.7103],
        [-0.8276,  1.0454],
        [ 3.8728, -0.3351]]) tensor([[ 0.5544,  0.4389],
        [ 0.5544,  0.4389],
        [ 0.6207, -0.7840]])
